In [ ]:
%pip install bibtexparser
%pip install vllm
!wget "https://evilscript.eu/upload/files/testtest.bib"

In [ ]:
import pickle

import bibtexparser
from vllm import LLM


def create_prompts(file: str) -> dict:
    with open(file) as bibtex_file:
        bib_database = bibtexparser.load(bibtex_file)    
    
    topics = ["2D/3D Rendering", "Character Generation", "Game Engines", "Modeling", "Texture Synthesis", "AI-Augmented Serious Games", "Artificial Life Simulation", "Autonomous Vehicles", "Dynamic Difficulty Adjustment", "Game AI Directors", "Game Analytics", "Cheating Detection", "Churn Prediction", "Victory Prediction", "Game Design Patterns", "GVGAI", "Motion Detection", "NPC & AI", "Agents & Intelligent Controllers", "Procedural Content Generation", "Recommender Systems", "Inventory Control", "Item Recommendation", "Routing / Pathfinding and Search", "Video game industry, bug and production", "VR/AR & AI", "Surveys on Video games" ]

    prompts = {
        "Given the following topics, categorize the presented paper (in Bibtex). Return only the categorization and no other output:\n" +
        "Topics:" + ", ".join(topics) + "\n" +
        "Paper: {}\n".format(entry['title']) +
        ('Abstract: ' + entry['abstract'] + '\n' if 'abstract' in entry else '') +
        ('Keywords: ' + entry['keywords'] + '\n' if 'keywords' in entry else '') +
        "Response: ": entry
        for entry in bib_database.entries
    }

    return prompts


if __name__ == "__main__":
    prompts = create_prompts("testtest.bib")
    # If there's a pickle file with the outputs, load it
    try:
        with open("outputs.pkl", "rb") as f:
            outputs = pickle.load(f)
    except FileNotFoundError:
        # Create a new LLM object
        llm = LLM(model="TheBloke/Yi-34B-AWQ", trust_remote_code=True)
        outputs = llm.generate([prompt for prompt in prompts.keys()])
        # Save outputs to local file using pickle
        with open("outputs.pkl", "wb") as f:
            pickle.dump(outputs, f)
     # Save the outputs to a txt file (name of the paper: classification)
    with open("outputs.txt", "w") as f:
        for prompt, output in zip(prompts.keys(), outputs):
            # output.outputs is a list
            f.write(prompts[prompt]['title'] + ": " + output.outputs[0].text + "\n")

In [ ]:
!curl --upload-file ./outputs.txt https://transfer.sh/outputs.txt

In [ ]:
!rm outputs.pkl